In [47]:
class Ten:
    def __get__(self, instance, owner):
        # self: Ten 的实例
        # instance: A 的实例
        # owner: class A
        print(f'instance:{instance}. owner: {owner}')
        return 10


class A:
    x = 5
    # 要使用描述器，它必须作为一个类变量存储在另一个类中：
    y = Ten()


if __name__ == '__main__':
    a = A()
    print(a.x)
    print(a.y)


5
instance:<__main__.A object at 0x11fcc1be0>. owner: <class '__main__.A'>
10


In [56]:
import logging

logging.basicConfig(level=logging.INFO)


class LoggingAgeAccess():
    def __get__(self, instance, owner):
        value = instance._age
        logging.info('Accessing %r giving %r', 'age', value)
        return value

    def __set__(self, obj, value):
        logging.info('Updating %r to %r', 'age', value)
        obj._age = value


class Person:
    age = LoggingAgeAccess()

    def __init__(self, name, age):
        self.name = name
        self.age = age

    def birthday(self):
        self.age += 1

    def get_age(self):
        print(Person.age)
        print(self.age)


if __name__ == '__main__':
    mary = Person('Mary M', 30)
    dave = Person('David D', 40)
    mary.birthday()

    print(f'vars(mary):{vars(mary)}')
    print(f'vars(dave):{vars(dave)}')
    print(f'dave.name: {dave.name}')
    print(f'dave.age:{dave.age}')

INFO:root:Updating 'age' to 30
INFO:root:Updating 'age' to 40
INFO:root:Accessing 'age' giving 30
INFO:root:Updating 'age' to 31
INFO:root:Accessing 'age' giving 40


vars(mary):{'name': 'Mary M', '_age': 31}
vars(dave):{'name': 'David D', '_age': 40}
dave.name: David D
dave.age:40


In [160]:
import logging

logging.basicConfig(level=logging.INFO)


class LoggedAccess:

    def __set_name__(self, owner, name):
        print(f'owner:{owner}. name:{name}')
        self.public_name = name
        self.private_name = '_' + name

    def __get__(self, instance, owner=None):
        value = getattr(instance, self.private_name)
        logging.info('Accessing %r giving %r', self.public_name, value)
        return value

    def __set__(self, instance, value):
        logging.info('Updating %r to %r', self.public_name, value)
        setattr(instance, self.private_name, value)


class Person:
    hobby = LoggedAccess()  # First descriptor instance
    age = LoggedAccess()  # Second descriptor instance

    def __init__(self, hobby, age):
        self.hobby = hobby  # Calls the first descriptor
        self.age = age  # Calls the second descriptor

    def birthday(self):
        self.age += 1

owner:<class '__main__.Person'>. name:hobby
owner:<class '__main__.Person'>. name:age


In [161]:
vars(vars(Person)['hobby'])

{'public_name': 'hobby', 'private_name': '_hobby'}

In [162]:
boy = Person('play football', 40)

INFO:root:Updating 'hobby' to 'play football'
INFO:root:Updating 'age' to 40


In [163]:
cls_type = type(boy)
cls_type

__main__.Person

In [164]:
vars(boy)

{'_hobby': 'play football', '_age': 40}

In [165]:
cls_var = getattr(cls_type, '_hobby', object())
cls_var

In [166]:
getattr(type(cls_var), '__get__', object())

In [167]:
boy.hobby

INFO:root:Accessing 'hobby' giving 'play football'


'play football'

In [168]:
# Validate
from abc import ABC, abstractmethod


class Validator(ABC):
    def __set_name__(self, owner, name):
        self.private_name = '_' + name

    def __get__(self, instance, owner):
        return getattr(instance, self.private_name)

    def __set__(self, instance, value):
        self.validate(value)
        setattr(instance, self.private_name, value)

    @abstractmethod
    def validate(self, value):
        pass

In [169]:
class Numbers(Validator):
    def __init__(self, minvalue=None, maxvalue=None):
        self.minvalue = minvalue
        self.maxvalue = maxvalue

    def validate(self, value):
        if not isinstance(value, (int, float)):
            raise TypeError(f'Expected {value!r} to be an int or float')
        if self.minvalue and value < self.minvalue:
            raise ValueError(f'Expected {value!r} to be at least {self.minvalue!r}')
        if self.maxvalue and value > self.maxvalue:
            raise ValueError(
                f'Expected {value!r} to be no more than {self.maxvalue!r}'
            )

In [170]:
class NumbersTest():
    length = Numbers(minvalue=18)

    def __init__(self, name, length):
        self.name = name
        self.length = length

In [171]:
NumbersTest('boxy', 20)

In [172]:
NumbersTest('boxy', 0)

ValueError: Expected 0 to be at least 18

In [ ]:
NumbersTest('boxy', '0')

In [ ]:
def object_getattribute(obj, name):
    null = 0
    # obj - 实例
    cls_type = type(obj)
    cls_var = getattr(cls_type, name, null)
    descr_get = getattr(type(cls_var), '__get__', null)
    print(f'cls_type:{cls_type},cls_var:{cls_var},descr_get:{descr_get}')
    if descr_get is not null and (
            hasattr(type(cls_var), '__set__') or hasattr(type(cls_var), '__delete__')
    ):
        print('数据描述器')
        return descr_get(cls_var, obj, cls_type)
    if hasattr(obj, '__dict__') and name in vars(obj):
        print('实例变量')
        return vars(obj)[name]
    if descr_get is not null:
        print('非数据描述器')
        return descr_get(cls_var, obj, cls_type)
    if cls_var is not null:
        print('类变量')
        return cls_var

    raise AttributeError(name)




In [ ]:
object_getattribute(boy, 'hobby')

In [ ]:
class C(object):
    def __init__(self):
        self._x = None

    @property
    def x(self):
        """I'm the 'x' property."""
        print("I'm the 'x' property.")
        return self._x

    @x.setter
    def x(self, value):
        print('set')
        self._x = value

    @x.deleter
    def x(self):
        print('del')
        del self._x

In [ ]:

class C(object):
    def __init__(self):
        self._x = None

    def get_x(self):
        """I'm the 'x' property."""
        print("I'm the 'x' property.")
        return self._x

    def set_x(self, value):
        print('set')
        self._x = value

    def del_x(self):
        print('del')
        del self._x

    x = property(get_x, set_x, del_x)

In [ ]:
c = C()
c.x = 123

In [ ]:
class persion:
    def __init__(self, name, age):
        self.name = name
        self.__age = age  #:age是一个私有属性

    def tell(self):
        print("我叫%s,我今年%s岁了" % (self.name, self.__age))


p = persion("杨洋", 21)
p.name = "小明"
print(p.name)  #:可以查看和修改

p.__age = 22  #:修改也不可以
# print(p.__age)      #:外部想去查看私有属性是查看不了的

In [ ]:
p.__age

In [ ]:
p.__age = 90

In [ ]:
p.__age

In [1]:
class Animal(object):
    def __init__(self, kind, name):
        self.kind = kind
        self._name = name

    def __getattr__(self, item):
        print(f'[__getattr__]items:{item}')

        if item == 'name':
            return self._name
        raise AttributeError(item)

    def __getattribute__(self, item):
        # print(self.kind)
        print(f'[__getattribute__]items:{item}')
        return object.__getattribute__(self, item)

In [2]:
dog1 = Animal('dog', '阿黄')

In [3]:
dog1.name

[__getattribute__]items:name
[__getattr__]items:name
[__getattribute__]items:_name


'阿黄'

In [4]:
dog1.no_exist_name


[__getattribute__]items:no_exist_name
[__getattr__]items:no_exist_name


AttributeError: no_exist_name

In [5]:
vars(dog1)

[__getattribute__]items:__dict__


{'kind': 'dog', '_name': '阿黄'}

In [6]:
dog1.kind

[__getattribute__]items:kind


'dog'

In [7]:
# 属性管理技术的比较

In [21]:
## property 实现
class Powers():
    def __init__(self, square, cube):
        self._square = square
        self._cube = cube

    def get_square(self):
        return self._square ** 2

    def set_square(self, value):
        self._square = value

    square = property(get_square, set_square)

    def get_cube(self):
        return self._cube ** 3

    cube = property(get_cube)


if __name__ == '__main__':
    p = Powers(3, 4)
    print(f"p.square:{p.square}")
    print(f"p.cube:{p.cube}")
    p.square = 5
    print("-" * 40)
    print(f"p.square:{p.square}")

p.square:9
p.cube:64
----------------------------------------
p.square:25


In [ ]:
## 描述器实现

import logging

logging.basicConfig(level=logging.INFO)


class Calculate(object):
    def __init__(self, factor=None):
        self.factor = factor

    def __set_name__(self, owner, name):
        self.public_name = name
        self.private_name = '_' + name

    def __get__(self, instance, owner):
        # if self.public_name == 'square':
        #     return getattr(instance, self.private_name) ** 2
        #
        # elif self.public_name == 'cube':
        #
        #     return getattr(instance, self.private_name) ** 3
        # else:
        #     raise AttributeError(self.public_name)

        if self.factor:
            return getattr(instance, self.private_name) ** 2
        else:
            raise AttributeError(self.factor)

    def __set__(self, instance, value):
        logging.info('Updating %r to %r', self.public_name, value)
        setattr(instance, self.private_name, value)


class Powers():
    square = Calculate(2)
    cube = Calculate(3)

    def __init__(self, square, cube):
        self.square = square
        self.cube = cube


if __name__ == '__main__':
    p = Powers(3, 4)
    print(f"p.square:{p.square}")
    print(f"p.cube:{p.cube}")
    p.square = 5
    print("-" * 40)
    print(f"p.square:{p.square}")

In [43]:
## __getattr__ 实现
class Powers():
    def __init__(self, square, cube):
        self._square = square
        self._cube = cube

    def __getattr__(self, item):
        """
        拦截实例不存在的属性
        :param item:
        :return:
        """
        if item == 'square':
            return self._square ** 2
        elif item == 'cube':
            return self._cube ** 3
        else:
            raise TypeError(f'unknow attr: {item}')

    def __setattr__(self, name, value):
        if name == 'square':
            self.__dict__['_square'] = value
        elif name == 'cube':
            self.__dict__['_cube'] = value

        else:
            self.__dict__[name] = value


if __name__ == '__main__':
    p = Powers(3, 4)
    print(f"p.square:{p.square}")
    print(f"p.cube:{p.cube}")
    p.square = 5
    print("-" * 40)
    print(f"p.square:{p.square}")

p.square:9
p.cube:64
----------------------------------------
p.square:25


In [45]:
## __getattribute__ 实现
class Powers():
    def __init__(self, square, cube):
        self._square = square
        self._cube = cube

    def __getattribute__(self, item):
        """
        拦截实例所有有属性，包括不存在的属性
        :param item:
        :return:
        """
        if item == 'square':
            return object.__getattribute__(self, '_square') ** 2
        elif item == 'cube':
            return object.__getattribute__(self, '_cube') ** 3
        else:
            return object.__getattribute__(self, item)

    def __setattr__(self, name, value):
        if name == 'square':
            self.__dict__['_square'] = value
        elif name == 'cube':
            self.__dict__['_cube'] = value

        else:
            self.__dict__[name] = value


if __name__ == '__main__':
    p = Powers(3, 4)
    print(f"p.square:{p.square}")
    print(f"p.cube:{p.cube}")
    p.square = 5
    print("-" * 40)
    print(f"p.square:{p.square}")
    print(vars(p))

p.square:9
p.cube:64
----------------------------------------
p.square:25
{'_square': 5, '_cube': 4}


In [63]:
class Animal(object):
    __slots__ = ['kind', '_name']

    def __init__(self, kind, name):
        self.kind = kind
        self._name = name

    def __getattr__(self, item):
        print(f'[__getattr__]items:{item}')

        if item == 'name':
            return self._name
        raise AttributeError(item)

    def __getattribute__(self, item):
        # print(self.kind)
        print(f'[__getattribute__]items:{item}')
        return object.__getattribute__(self, item)

    def __str__(self):
        print()


if __name__ == '__main__':
    dog = Animal('dog', 'dingding')
    dog.my_slot = 'slot_test'
    print(dog.my_slot)


AttributeError: 'Animal' object has no attribute 'my_slot'

In [64]:
line_list = ['  line 1\n', 'line 2  \n', ...]

# Generator expression -- returns iterator
stripped_iter = (line.strip() for line in line_list)

# List comprehension -- returns list
stripped_list = [line.strip() for line in line_list]

AttributeError: 'ellipsis' object has no attribute 'strip'

In [5]:
class TestItem():
    def __init__(self, mytest):
        self.mytest = mytest

    def __getattr__(self, item):
        print(f"__getattr__:{item}")
        return getattr(self.mytest, item)

    def __getitem__(self, item):
        print(f"__getitem__:{item}")
        return self.mytest[item]

    def __setitem__(self, key, value):
        print(f"__setitem__:{key}")
        self.mytest[key] = value


t = TestItem(123123)
t


__getattr__:_ipython_canary_method_should_not_exist_
__getattr__:_ipython_display_
__getattr__:_ipython_canary_method_should_not_exist_
__getattr__:_repr_mimebundle_
__getattr__:_ipython_canary_method_should_not_exist_
__getattr__:_repr_html_
__getattr__:_ipython_canary_method_should_not_exist_
__getattr__:_repr_markdown_
__getattr__:_ipython_canary_method_should_not_exist_
__getattr__:_repr_svg_
__getattr__:_ipython_canary_method_should_not_exist_
__getattr__:_repr_png_
__getattr__:_ipython_canary_method_should_not_exist_
__getattr__:_repr_pdf_
__getattr__:_ipython_canary_method_should_not_exist_
__getattr__:_repr_jpeg_
__getattr__:_ipython_canary_method_should_not_exist_
__getattr__:_repr_latex_
__getattr__:_ipython_canary_method_should_not_exist_
__getattr__:_repr_json_
__getattr__:_ipython_canary_method_should_not_exist_
__getattr__:_repr_javascript_


In [6]:
from functools import wraps
def trace(func):
    # @wraps(func)
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        print(f'{func.__name__}({args!r},{kwargs!r}) -> {result!r}')
        return result

    return wrapper

@trace
def fibonacci(n):
    """test"""
    if n in (0, 1):
        return n
    return (fibonacci(n - 2) + fibonacci(n - 1))

fibonacci(4)
help(fibonacci)
print(type(fibonacci))

fibonacci((0,),{}) -> 0
fibonacci((1,),{}) -> 1
fibonacci((2,),{}) -> 1
fibonacci((1,),{}) -> 1
fibonacci((0,),{}) -> 0
fibonacci((1,),{}) -> 1
fibonacci((2,),{}) -> 1
fibonacci((3,),{}) -> 2
fibonacci((4,),{}) -> 3
Help on function wrapper in module __main__:

wrapper(*args, **kwargs)
    # @wraps(func)

<class 'function'>
